<a href="https://colab.research.google.com/github/MahdiTheGreat/Intro-to-language-modeling/blob/main/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipdb
!pip install -U spacy
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 925.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 45.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Unin

In [2]:
import spacy
import torch
import matplotlib.pyplot as plt
import ipdb
import numpy as np
import random
import pandas as pd
# %pdb on

In [3]:
# Helper function to plot the training metrics

def plot_training_metrics(train_acc, val_acc, train_loss, title, save_path):
    # Ensure that all input lists have the same length
    assert len(train_acc) == len(val_acc) == len(train_loss), "All input histories must have the same length."

    epochs = range(1, len(train_acc) + 1)

    # Create the metrics DataFrame
    df_metrics = pd.DataFrame({
        'Epoch': epochs,
        'Training Accuracy (%)': train_acc,
        'Validation Accuracy (%)': val_acc,
        'Training Loss': train_loss
    })

    # Initialize the plot
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot Training and Validation Accuracy on ax1
    color = 'tab:blue'
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy (%)', color=color)
    ax1.plot(df_metrics['Epoch'], df_metrics['Training Accuracy (%)'], label='Train Acc', color='tab:blue')
    ax1.plot(df_metrics['Epoch'], df_metrics['Validation Accuracy (%)'], label='Val Acc', color='tab:cyan')
    ax1.tick_params(axis='y', labelcolor=color)

    # Create a second y-axis for Training Loss
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Loss', color=color)
    ax2.plot(df_metrics['Epoch'], df_metrics['Training Loss'], label='Train Loss', color='tab:red')
    ax2.tick_params(axis='y', labelcolor=color)

    # Combine legends from both axes
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()
    ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left')

    # Set plot title and layout
    plt.title(title)
    plt.tight_layout()

    # Save and display the plot
    plt.savefig(save_path)
    plt.show()

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim

# EarlyStopping class remains the same
class EarlyStopping:
    def __init__(self, patience=5, delta=0, verbose=False, path='checkpoint.pth'):
        self.patience = patience  # Number of epochs to wait for improvement
        self.delta = delta  # Minimum change to qualify as an improvement
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.path = path  # Path to save the best model

    def __call__(self, val_loss, model):
        if self.best_score is None:
            self.best_score = val_loss
            self.save_checkpoint(val_loss, model)
        elif val_loss < self.best_score - self.delta:
            self.best_score = val_loss
            self.save_checkpoint(val_loss, model)
            self.counter = 0
        else:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True

    def save_checkpoint(self, val_loss, model):
        '''Save model when validation loss decreases.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

class SimpleANN(nn.Module):

    def __init__(self, vocab_size, embed_size, layer_sizes,activation=nn.ReLU,last_layer_activation=nn.Softmax,dropout=0):

        super(SimpleANN, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embed_size)
        self.layers = nn.ModuleList()

        for i in range(len(layer_sizes)-2):
          self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
          self.layers.append(nn.Dropout(dropout))
          self.layers.append(activation())

        self.layers.append(nn.Linear(layer_sizes[-2], layer_sizes[-1]))
        if last_layer_activation is not None:
         self.layers.append(nn.Dropout(dropout))
         self.layers.append(last_layer_activation())

    def forward(self, x):
        # Assuming x is a batch of word indices (e.g., [batch_size])
        embeddings = self.embeddings(x)  # Get word embeddings for each word in the batch

        # Flatten the input embeddings (if necessary, depending on your task)
        x = embeddings.view(-1, np.prod(embeddings.shape[1:]))  # Flatten for fully connected layers

        #x = x.view(-1, np.prod(x.shape[1:])) # Flatten the input
        x = x.float()
        for layer in self.layers:
            x = layer(x)
        return x

In [5]:
# Set random seed for reproducibility
def set_seed(seed=2024):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed(1998)

In [6]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))
print(f'Using device: {device}')

Using device: cpu


In [7]:
from copy import deepcopy
def dense_arch_builder(input_size,scale_factor=0,hidden_layers_num=0,repeat=0,output_size=1):
  layer_sizes=[input_size]

  if scale_factor!=0:

   if scale_factor>1:
    for i in range(hidden_layers_num):
     layer_sizes.append(layer_sizes[-1]*scale_factor)
    while layer_sizes[-1]<output_size:
     layer_sizes.append(layer_sizes[-1]*scale_factor)

   elif scale_factor==1:
     for i in range(2,hidden_layers_num+2):
      layer_sizes.append(layer_sizes[0]*i)
     i+=1
     while layer_sizes[-1]<output_size:
      layer_sizes.append(layer_sizes[0]*i)
      i+=1

   mirrored_layer_sizes=deepcopy(layer_sizes)
   mirrored_layer_sizes.reverse()
   mirrored_layer_sizes=mirrored_layer_sizes[1:-1]

   for i in range(repeat):
    layer_sizes.append(layer_sizes[-1])

   if output_size>0:
    layer_sizes+=mirrored_layer_sizes
    downscale_factor=scale_factor if scale_factor>1 else 2

    while layer_sizes[-1]!=output_size:
     if layer_sizes[-1]//downscale_factor>=output_size:
      layer_sizes.append(layer_sizes[-1]//downscale_factor)
     else:
      layer_sizes.append(output_size)

  else:
    downscale_factor = (input_size / output_size) ** (1 / hidden_layers_num)
    for i in range(hidden_layers_num):
        layer_sizes.append(int(input_size / (downscale_factor ** i)))
    layer_sizes.append(output_size)

  return layer_sizes


In [37]:
layer_sizes=dense_arch_builder(input_size=3,scale_factor=2,hidden_layers_num=4,repeat=0,output_size=24)
print(layer_sizes)

[3, 6, 12, 24, 48, 24, 12, 6, 24]


In [40]:
## Creating a tensor dataset ##
from torch.utils.data import DataLoader, TensorDataset
def TorchDataLoader(training_sequences, batch_size):
  context_words = [item[0] for item in training_sequences]  # List of [context]
  target_words = [item[1] for item in training_sequences]   # List of target words

  # Convert lists to tensors
  context_tensor = torch.tensor(context_words, dtype=torch.long)  # Shape: (num_samples, 3)
  target_tensor = torch.tensor(target_words, dtype=torch.long)    # Shape: (num_samples,)

  # Create a TensorDataset
  dataset = TensorDataset(context_tensor, target_tensor)

  # Create a DataLoader for batching
  batch_size = 4
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

  # Iterate over the DataLoader
  #for batch_context, batch_target in dataloader:
  #    print("Batch context:", batch_context)
  #    print("Batch target:", batch_target)
      # You can now use batch_context and batch_target for model training

  return dataloader

In [48]:
dummy_text = [
    ([2, 2, 2], 7), ([2, 2, 7], 8), ([2, 7, 8], 3), ([7, 8, 3], 9),
    ([8, 3, 9], 4), ([3, 9, 4], 5), ([9, 4, 5], 2), ([2, 2, 2], 2),
    ([2, 2, 2], 12), ([2, 2, 12], 4), ([2, 12, 4], 5), ([12, 4, 5], 13),
    ([4, 5, 13], 3), ([5, 13, 3], 14), ([13, 3, 14], 15), ([3, 14, 15], 2),
    ([2, 2, 2], 6), ([2, 2, 6], 16), ([2, 6, 16], 17), ([6, 16, 17], 18),
    ([16, 17, 18], 19), ([17, 18, 19], 20), ([18, 19, 20], 6), ([19, 20, 6], 21),
    ([20, 6, 21], 22), ([6, 21, 22], 2)
]

training_size = 0.8
training_text = dummy_text[:int(len(dummy_text)*training_size)]
val_text = dummy_text[int(len(dummy_text)*training_size):]

trainloader = TorchDataLoader(training_text, 4)
valloader = TorchDataLoader(val_text, 4)

In [69]:
layer_sizes=dense_arch_builder(input_size=72,scale_factor=2,hidden_layers_num=4,repeat=0,output_size=24)
print(layer_sizes)

model = SimpleANN(layer_sizes=layer_sizes, vocab_size=24, embed_size=24)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

patience = 5
early_stopping = EarlyStopping(patience=patience, verbose=True)

number_of_epochs = 30

for epoch in range(number_of_epochs):
    for batch_context, batch_target in trainloader:
        #FORWARD PASS:
        X = batch_context
        Y = batch_target
        X, Y = X.to(device), Y.to(device)
        outputs = model(X)  # Model output for X
        loss = criterion(outputs, Y) # Compute the loss between model output and Y

        #BACKWARD PASS (updating the model parameters):
        optimizer.zero_grad()  # Clear gradients
        loss.backward()        # Compute gradients
        optimizer.step()       # Update model parameters

    print(f"Epoch [{epoch+1}/{number_of_epochs}], Loss: {loss.item():.4f}")

    # Validation loop
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():  # No gradient computation for validation
        for inputs, targets in valloader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(valloader)  # Average validation loss
    print(f"Epoch {epoch+1}/{number_of_epochs} - Validation Loss: {avg_val_loss:.6f}")

    # Call early stopping after each epoch
    early_stopping(avg_val_loss, model)

    if early_stopping.early_stop:
        print("Early stopping triggered!")
        break

# Optionally, load the best model after training
model.load_state_dict(torch.load('checkpoint.pth'))

[72, 144, 288, 576, 1152, 576, 288, 144, 72, 36, 24]


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch [1/30], Loss: 3.1813
Epoch 1/30 - Validation Loss: 3.181040
Validation loss decreased (inf --> 3.181040).  Saving model ...
Epoch [2/30], Loss: 3.1784
Epoch 2/30 - Validation Loss: 3.183803
EarlyStopping counter: 1 out of 5
Epoch [3/30], Loss: 3.1586
Epoch 3/30 - Validation Loss: 3.179778
Validation loss decreased (3.181040 --> 3.179778).  Saving model ...
Epoch [4/30], Loss: 3.2468
Epoch 4/30 - Validation Loss: 3.247109
EarlyStopping counter: 1 out of 5
Epoch [5/30], Loss: 2.9968
Epoch 5/30 - Validation Loss: 3.234914
EarlyStopping counter: 2 out of 5
Epoch [6/30], Loss: 3.2173
Epoch 6/30 - Validation Loss: 3.208758
EarlyStopping counter: 3 out of 5
Epoch [7/30], Loss: 3.0027
Epoch 7/30 - Validation Loss: 3.169878
Validation loss decreased (3.179778 --> 3.169878).  Saving model ...
Epoch [8/30], Loss: 2.9398
Epoch 8/30 - Validation Loss: 3.193126
EarlyStopping counter: 1 out of 5
Epoch [9/30], Loss: 3.1645
Epoch 9/30 - Validation Loss: 3.222477
EarlyStopping counter: 2 out of 5


<ipython-input-69-c2b3ad6dad48>:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('checkpoint.pth'))


<All keys matched successfully>

# Step 4

Predicting the next word for four sentences:

In [70]:
test_sentences = torch.tensor([[0, 13, 12],
                              [0, 8, 9],
                              [8, 7, 6],
                              [5, 4, 5]])
output = model(test_sentences).detach().numpy()

# Predict
predictions = np.argmax(output, axis=1)

print(predictions)

[2 2 2 2]


Quantitative evaluation

In [71]:
# Predict validation data
dummy_val = [
    ([2, 2, 2], 7), ([2, 2, 7], 8), ([2, 7, 8], 3), ([7, 8, 3], 9),
    ([8, 3, 9], 4), ([3, 9, 4], 5), ([9, 4, 5], 2), ([2, 2, 2], 2),
    ([2, 2, 2], 12), ([2, 2, 12], 4), ([2, 12, 4], 5), ([12, 4, 5], 13),
    ([4, 5, 13], 3), ([5, 13, 3], 14), ([13, 3, 14], 15), ([3, 14, 15], 2),
    ([2, 2, 2], 6), ([2, 2, 6], 16), ([2, 6, 16], 17), ([6, 16, 17], 18),
    ([16, 17, 18], 19), ([17, 18, 19], 20), ([18, 19, 20], 6), ([19, 20, 6], 21),
    ([20, 6, 21], 22), ([6, 21, 22], 2)
]
val_dataloader = TorchDataLoader(dummy_val, 4)
loss = []
for batch_context, batch_target in val_dataloader:
        #FORWARD PASS:
        X = batch_context
        Y = batch_target
        X, Y = X.to(device), Y.to(device)
        outputs = (model(X))  # Model output for X
        loss.append((criterion(outputs, Y)).item()) # Compute the loss between model output and Y

# Compute perplexity
perplexity = np.exp(np.mean(loss))
print(perplexity)

21.860031353068354


Inspecting the word embeddings

In [72]:
def nearest_neighbors(emb, voc, inv_voc, word, n_neighbors=5):

    # Look up the embedding for the test word.
    test_emb = emb.weight[voc[word]]

    # We'll use a cosine similarity function to find the most similar words.
    sim_func = nn.CosineSimilarity(dim=1)
    cosine_scores = sim_func(test_emb, emb.weight)

    # Find the positions of the highest cosine values.
    near_nbr = cosine_scores.topk(n_neighbors+1)
    topk_cos = near_nbr.values[1:]
    topk_indices = near_nbr.indices[1:]
    # NB: the first word in the top-k list is the query word itself!
    # That's why we skip the first position in the code above.

    # Finally, map word indices back to strings, and put the result in a list.
    return [ (inv_voc[ix.item()], cos.item()) for ix, cos in zip(topk_indices, topk_cos) ]
word_embedding = model.embeddings(torch.tensor([5]))
print(word_embedding)
nearest_neighbors(word_embedding)
nearest_neighbors("2005")

tensor([[ 2.8097,  0.6443, -0.6667,  0.2803, -0.7137, -0.7417,  0.5777, -0.0141,
         -0.9880, -2.3758, -0.8092,  0.3889, -1.6978,  1.1951,  0.6678,  1.4581,
          0.9677, -0.4329, -1.4000,  1.2658,  0.6585,  0.7534, -1.5279, -0.8618]],
       grad_fn=<EmbeddingBackward0>)


TypeError: nearest_neighbors() missing 3 required positional arguments: 'voc', 'inv_voc', and 'word'

In [73]:
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
def plot_embeddings_pca(emb, inv_voc, words):
    vectors = np.vstack([emb.weight[inv_voc[w]].cpu().detach().numpy() for w in words])
    vectors -= vectors.mean(axis=0)
    twodim = TruncatedSVD(n_components=2).fit_transform(vectors)
    plt.figure(figsize=(5,5))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.02, y, word)
    plt.axis('off')

plot_embeddings_pca(model[0], prepr, ['sweden', 'denmark', 'europe', 'africa', 'london', 'stockholm', 'large', 'small', 'great', 'black', '3', '7', '10', 'seven', 'three', 'ten', '1984', '2005', '2010'])

/usr/local/lib/python3.10/dist-packages/numpy/_core/_dtype.py:106: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if dtype.type == np.bool:


TypeError: 